## Imports

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, col, trim
from pyspark.sql.types import DecimalType
from conexao_bancodedados import connection_mysql, properties_mysql, write_pyspark_mysql

## Start Spark Session

In [ ]:
# Criação da SparkSession
spark = SparkSession.builder \
    .appName("Leitura e Transformação MySQL") \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

## Variables

In [ ]:
# Configurações para conexão com o banco de dados MySQL
jdbc_url = connection_mysql("bronze")
jdbc_properties = properties_mysql()

## Read Table

In [ ]:
# Lendo a tabela MySQL no DataFrame do PySpark
df = spark.read.jdbc(url=jdbc_url, table='cotahist', properties=jdbc_properties)

## Transformation

In [ ]:


df_transformado = df.select(
    to_date(col("data_pregao"), "yyyyMMdd").alias("data_pregao"),
    trim(col("codigo_negociacao")).alias("codigo_negociacao"),
    (col("preco_abertura_papel").cast("float") / 100).alias("preco_abertura_papel"),
    (col("preco_maximo_papel").cast("float") / 100).alias("preco_maximo_papel"),
    (col("preco_minimo_papel").cast("float") / 100).alias("preco_minimo_papel"),
    (col("preco_medio_papel").cast("float") / 100).alias("preco_medio_papel"),
    (col("preco_ultimo_negocio").cast("float") / 100).alias("preco_ultimo_negocio"),
    (col("preco_melhor_oferta_compra").cast("float") / 100).alias("preco_melhor_oferta_compra"),
    (col("preco_melhor_oferta_venda").cast("float") / 100).alias("preco_melhor_oferta_venda"),
    (col("volume_total_titulos").cast("float") / 100).alias("volume_total_titulos"),
    (col("preco_exercicio_opcoes").cast("float") / 100).alias("preco_exercicio_opcoes"),
    (col("preco_exercicio_pontos").cast(DecimalType(7,6)) / 1000000).alias("preco_exercicio_pontos")
)


In [ ]:
# Transformação dos dados
""" df_transformado = df.withColumn("data_pregao", to_date(col("data_pregao"), "yyyyMMdd")) \
                    .withColumn("codigo_negociacao", trim(col("codigo_negociacao"))) \
                    .withColumn("preco_abertura_papel", col("preco_abertura_papel").cast("float") / 100) \
                    .withColumn("preco_maximo_papel", col("preco_maximo_papel").cast("float") / 100) \
                    .withColumn("preco_minimo_papel", col("preco_minimo_papel").cast("float") / 100) \
                    .withColumn("preco_medio_papel", col("preco_medio_papel").cast("float") / 100) \
                    .withColumn("preco_ultimo_negocio", col("preco_ultimo_negocio").cast("float") / 100) \
                    .withColumn("preco_melhor_oferta_compra", col("preco_melhor_oferta_compra").cast("float") / 100) \
                    .withColumn("preco_melhor_oferta_venda", col("preco_melhor_oferta_venda").cast("float") / 100) \
                    .withColumn("volume_total_titulos", col("volume_total_titulos").cast("float") / 100) \
                    .withColumn("preco_exercicio_opcoes", col("preco_exercicio_opcoes").cast("float") / 100) \
                    .withColumn("preco_exercicio_pontos", col("preco_exercicio_pontos").cast(DecimalType(7, 6)) / 1000000) """


## Save Data Frame

In [ ]:
write_pyspark_mysql(df, 'cotahist', 'bronze', 'overwrite')

## Stop Spark Session

In [ ]:
# Encerra a SparkSession
spark.stop()